Mission to Mars

10.3.3 Scrape Mars Data:

set up dependencies and the URL executable path

In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

#using pandas for the .read_html() function
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 89.0.4389
Get LATEST driver version for 89.0.4389
Driver [C:\Users\phili\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


include the following optional delay line for 2 reasons:

browser.is_element_present_by_css('div.list_text', wait_time=1)

1. the search is for elements with a specific combination of tag (div) and attribute (list_text). As an example, ul.item_list would be found in HTML as <ul class="item_list">.

    
2. it tellls the browser to wait one second before searching for components. The optional delay is useful because sometimes dynamic pages take a little while to load, especially if they are image-heavy.

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

Set up the HTML parser

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

slide_elem has been set to hold all the data that should be searched.  

find the data "content_title" in the slide_elem variable.

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Mars 2020 Stands on Its Own Six Wheels</div>

Remove the HTML code comments.  

Do this by chaining .get_text() to the .find() method.  then only the text of the element is returned.

Note:
There are two methods used to find tags and attributes with BeautifulSoup:

- .find() is used when we want only the first class and attribute we've specified.


-  .find_all() is used when we want to retrieve all of the tags and attributes.


For example, if we were to use .find_all() instead of .find() when pulling the summary, we would retrieve all of the summaries on the page instead of just the first one.


In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Mars 2020 Stands on Its Own Six Wheels'

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"In time-lapse video, taken at JPL, captures the first time NASA's Mars 2020 rover carries its full weight on its legs and wheels."

Mod_10.3.4  Scrape Mars Data:


### Featured Images


In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

chain some indexing at the end of .find to stipulate the 2nd button (remember(0,1,2)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

Parse the newly opened page

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

Find the relative url of the image.  its relative as the image will change with each new story.  the returned URL is only for the image, not the site.

In [11]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

Use an f string to add the base URL to the relative url derived above to get the complete url of the pictures to be scraped.

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

10.3.5 Scrape Mars Data: Just the facts Ma'am


Pull data from table format 

Using pandas to read the html code and set it into a dataframe.

the index [0] calls for only the first table to be found



In [13]:
# read the first table to be found
df = pd.read_html('https://galaxyfacts-mars.com')[0]

# assign column headers
df.columns=['description', 'Mars', 'Earth']

#turn the description column into the index and maintain it without having to reassign the variable.
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [15]:
browser.quit()

Mod_10.3.6 Export to Python

Jupyter notebooks can't be automated for scraping.  To fully automate the code, it will need to be converted into a .py file.

The next step in making this an automated process is to download the current code into a Python file. It won't transition over perfectly, so it will need to be cleaned up a bit, but it's an easier task than copying each cell and pasting it over in the correct order.